# Samples for Vertex AI Vector Search

### Chat bot scenario - Using User questions to determine service category

reference https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/intro-textemb-vectorsearch.ipynb

#### Generate test data
List of json formatted input 

```
{ 'qn':'I would like to change my delivery date', 'cat': '1'},
{ 'qn': 'change delivery address', 'cat': '1'},
{ 'qn': 'I would like to delay the delivery as I will be away', 'cat': '1'},
{ 'qn': 'Can I check the status of order #12345? it has been a while', 'cat': '2'},
{ 'qn': 'How long will order #483746 take? it has been 3 weeks since i ordered', 'cat': '2'},
{ 'qn': 'I would like to check my order status, the order number is 234555', 'cat': '2'},
{ 'qn': 'Can i get a refund , the product is defective', 'cat': '3'},
{ 'qn': 'What is the return process for a spoilt item?', 'cat': '3'},
{ 'qn': 'I got a wrong order, how do i do an exchange?', 'cat': '3'},
{ 'qn': 'The items i received is totally not working, please give me a refund', 'cat': '3'}
```


In [1]:
import pandas as pd
from typing import List, Optional


In [2]:
# load jsonl file 
jsonObj = pd.read_json(path_or_buf="questions-categories.jsonl", lines=True)
#print(jsonObj.qn[0])
#print(jsonObj.qn.to_string(index=False))
texts: List[str] = [jsonObj.qn.tolist]
print(texts)



[<bound method IndexOpsMixin.tolist of 0              I would like to change my delivery date
1                              change delivery address
2    I would like to delay the delivery as I will b...
3    Can I check the status of order #12345? it has...
4    How long will order #483746 take? it has been ...
5    I would like to check my order status, the ord...
6        Can i get a refund , the product is defective
7        What is the return process for a spoilt item?
8        I got a wrong order, how do i do an exchange?
9    The items i received is totally not working, p...
Name: qn, dtype: object>]


In [11]:
# sandbox-project-1673797795
# init the vertexai package
import vertexai
PROJECT_ID="sandbox-project-1673797795"
LOCATION="asia-southeast1"
vertexai.init(project=PROJECT_ID, location=LOCATION, service_account="953640930424-compute@developer.gserviceaccount.com")


In [13]:
# Load the text embeddings model
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel
model_name= "text-embedding-004",
task= "RETRIEVAL_DOCUMENT",
model = TextEmbeddingModel.from_pretrained(model_name)
# dimensionality: Optional[int] = 256,
# inputs = [TextEmbeddingInput(text, task) for text in texts]
# kwargs = dict(output_dimensionality=dimensionality) if dimensionality else {}
# embeddings = model.get_embeddings(inputs, **kwargs)


TypeError: can only concatenate str (not "tuple") to str